# obt_brasileirao_matches

Esse notebook é responsável por criar uma tabela com as informações descritivas e fatos de todas as partidas do Campeonato Brasileiro.

## Configurações Iniciais

### Importando as Bibliotecas


In [0]:
from pyspark.sql import functions as F

### Carregando os Dados

In [0]:
df_brasileirao_matches = (
    spark.read.csv(
        "/Volumes/workspace/default/data/Brasileirao_Matches.csv",
        header=True,
        inferSchema=True
    )
    .filter(
        (F.col("home_goal") != "NA")
        & (F.col("away_goal") != "NA")
    )
    .drop("home_team_state", "away_team_state")
)

## Tratamento dos Dados

In [0]:
df_final = (
    df_brasileirao_matches
    .withColumn(
        "championship",
        F.lit("Campeonato Brasileiro")
    )
    .withColumns({
        "home_goal": F.col("home_goal").cast("int"),
        "away_goal": F.col("away_goal").cast("int"),
    })
    .withColumn(
        "composite_key",
        F.concat_ws(
            "-",
            F.col("home_team"),
            F.col("away_team"),
            F.col("season"),
            F.col("round")
        )
    )
    .orderBy(F.desc("datetime"))
)

## Salvamento dos Dados

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ifrs_soccer.gold.obt_brasileirao_matches (
    datetime TIMESTAMP COMMENT "Data e hora da partida",
    home_team STRING COMMENT "Nome do time da casa",
    away_team STRING COMMENT "Nome do time visitante",
    home_goal INT COMMENT "Gols do time da casa",
    away_goal INT COMMENT "Gols do time visitante",
    season INT COMMENT "Temporada (ano)",
    round INT COMMENT "Rodada do campeonato",
    championship STRING COMMENT "Nome do campeonato",
    composite_key STRING PRIMARY KEY COMMENT "Chave composta única da partida formada por: home_team-away_team-season-round"
)
USING DELTA
TBLPROPERTIES ("quality"="gold", "layer"="data-engineering", "delta.enableTypeWidening"="true")
COMMENT "Tabela OBT com os resultados das partidas do Campeonato Brasileiro.";

In [0]:
(
    df_final
    .write.mode("overwrite")
    .option("mergeSchema", True)
    .saveAsTable("ifrs_soccer.gold.obt_brasileirao_matches")
)